In [1]:
import findspark
findspark.init()
# Creating Spark Context
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext("local", "first app")
spark = SparkSession(sc)
import plotly.graph_objects as go
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [2]:
text_file = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', quote='"', delimiter=',').load("C:\\Users\\mini_\\Desktop\Practica-Semi2\\archivos\\GuatemalaExportsTo.csv")
rddfiltro = text_file.rdd.map(tuple)

In [3]:
#se toman los aeropuertos y aterrizajes 
rddEXPORTS= rddfiltro.map(lambda word: (word[4],word[1]))
rddEXPORTS.take(10)

[('China', 18490000.0),
 ('China', 9830000.0),
 ('China', 8360000.0),
 ('China', 7110000.0),
 ('China', 4070000.0),
 ('China', 2590000.0),
 ('China', 2140000.0),
 ('China', 2130000.0),
 ('China', 2020000.0),
 ('China', 965560.0)]

In [5]:
#se suman todas las exportaciones reduciendo por la clave pais
rddCONT=rddEXPORTS.reduceByKey(lambda a,b: a+b)
print("Conteo total -> %s" % rddCONT.collect())

Conteo total -> [('China', 64176766.0), ('Mexico', 511215129.0), ('Canada', 222575942.0), ('Japon', 164579976.0), ('United Kingdom', 101910872.0), ('Costa Rica', 428419775.0), ('Saudi Arabia', 111274535.0), ('Gabon', 6420.0), ('Italy', 166680831.0), ('France', 34023574.0), ('Brazil', 39146101.0), ('Germany', 144422819.0), ('Honduras', 967311616.0), ('Venezuela', 14998293.0), ('El Salvador', 1217138626.0), ('United States', 3803123168.0)]


In [7]:
#se ordenan los valores y se toma el mayor
rddORDEN= sc.parallelize(rddCONT.sortBy(lambda a: a[1],False).take(1))
print("Mayor -> %s"%rddORDEN.collect())

Mayor -> [('United States', 3803123168.0)]


In [8]:
#se separan los nombre de los aeropuertos y totales
rddNombres = rddORDEN.map(lambda x: (x[0]))
rddTotales = rddORDEN.map(lambda x: (x[1]))
print(rddNombres.collect())
print(rddTotales.collect())

['United States']
[3803123168.0]


In [10]:
#se realiza la gráfica de barras
fig = go.Figure(data=go.Bar(x=rddNombres.collect(),y=rddTotales.collect()))
fig.update_layout(title_text='Pais con mayor exportaciones',title_font_size=30,
                  yaxis=dict(title='Total de exportaciones',title_font_size=25),
                  xaxis=dict(title='Nombre Pais',title_font_size=25))
fig.update_traces(overwrite=True, marker={"opacity": 0.5})
fig.write_html('R1_A1.html', auto_open=True)